In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

Installed package MathNet.Numerics.FSharp version 4.15.0

# Частичная проблема собственных значений
## Варинат 9

In [ ]:
let a = matrix [
    [-0.90701; -0.27716; 0.44570]
    [-0.27716; 0.63372; 0.07774]
    [0.44570; 0.07774; -0.95535]
]

**Задание 2.** Найти степенным методом c точностью ε = 0.001 максимальное по модулю собственное число λ матрицы A и соответствующий ему собственный вектор x, так чтобы ||x||_2 = 1

In [ ]:
let findEigenIterative accuracy (matrix: Matrix<float>) step = 
    // firstEstimation.[0] не должен быть слишком мал
    let firstEstimation = DenseVector.randomStandard<float> a.RowCount
    firstEstimation.[0] <- 1.
    
    let accurateLambda = matrix.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real

    let calculatePosteriorEstimation (eigenValueEstimation: float) (eigenVectorEstimation: Vector<float>) = 
        (matrix * eigenVectorEstimation - eigenValueEstimation * eigenVectorEstimation).L2Norm() /
        eigenVectorEstimation.L2Norm()
    
    let rec loop (previousVector: Vector<float>) iterationTable = 
        // unnormalized
        let y = matrix * previousVector
        let lambda = step y previousVector
        let updatedTable = 
            {|
                K = List.length iterationTable + 1
                Lambda = lambda
                AbsFactError = lambda - accurateLambda |> abs
                AYminusLambdaY = (matrix * y - lambda * y).L2Norm()
                PosteriorEstimation = calculatePosteriorEstimation lambda y
            |}
            |> List.singleton
            |> List.append iterationTable
        
        if calculatePosteriorEstimation lambda y >= accuracy then   
            loop y updatedTable
        else 
            lambda, (y / y.L2Norm()), updatedTable

    loop firstEstimation []

In [ ]:
let findEigenPowerMethod accuracy (matrix: Matrix<float>) = 
    findEigenIterative accuracy matrix <| fun (currentVector: Vector<float>) (previousVector: Vector<float>) ->
        // i любое
        (Vector.map2 (/) currentVector previousVector).[0]

In [ ]:
let epsilon = 0.001
let (eigenValuePower, eigenVectorPower, iterationTablePower) = findEigenPowerMethod epsilon a
printfn "Наибольшее по модулю собственное число, полученное степенным методом: %f" eigenValuePower
printfn "Соответствующий ему собственный вектор: %O" eigenVectorPower
printfn "Фактическая погрешнсть: %f" <| abs (eigenValuePower - a.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real)
iterationTablePower

Наибольшее по модулю собственное число, полученное степенным методом: 

-1.408068

Соответствующий ему собственный вектор: 

DenseVector 3-Double
-0,696668
 -0,12117
 0,707087


Фактическая погрешнсть: 

0.000288

index,AYminusLambdaY,AbsFactError,K,Lambda,PosteriorEstimation
0,0.32341895228349876,0.7855290780343804,1,-0.6222505366395275,0.42177150605757036
1,0.23671495124978,0.8644004130610661,2,-0.5433792016128418,0.5544545647355059
2,0.21621976001847135,0.5119524422028473,3,-0.8958271724710606,0.6988690234385924
3,0.20122244534937045,0.5067562399326545,4,-0.9010233747412534,0.5941615135202819
4,0.11667786794653631,0.11681475601151137,5,-1.2909648586623965,0.2578967826080197
5,0.10831713821404583,0.1332321742375091,6,-1.2745474404363988,0.1716441457447561
6,0.04942722876324101,0.00562692680534127,7,-1.4021526878685666,0.05573121017349013
7,0.045196116182672545,0.024854089430455417,8,-1.3829255252434525,0.03621128070136017
8,0.02292663439921462,0.0022241094940476103,9,-1.4100037241679555,0.013049052355910532
9,0.019154098797852733,0.004654795706023229,10,-1.4031248189678847,0.007744126617974683


**Задание 3.** Найти методом скалярных произведений с точностью ε^2 = 0.000001 максимальное по модулю собственное число матрицы λ. Сравнить требуемое количество итераций с п.2.

In [ ]:
let findEigenScalarMethod accuracy (matrix: Matrix<float>) = 
    findEigenIterative accuracy matrix <| fun (currentVector: Vector<float>) (previousVector: Vector<float>) ->
        // в предположении, что матрица симметрична
        (currentVector * previousVector) / (previousVector * previousVector)

In [ ]:
let epsilon2 = 0.001 ** 2.
let (eigenValueScalar, eigenVectorScalar, iterationTableScalar) = findEigenScalarMethod epsilon2 a
printfn "Наибольшее по модулю собственное число, полученное методом скалярных произведений: %f" eigenValueScalar
printfn "Соответствующий ему собственный вектор: %O" eigenVectorScalar
printfn "Фактическая погрешнсть: %f" <| abs (eigenValueScalar - a.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real)
iterationTableScalar

Наибольшее по модулю собственное число, полученное методом скалярных произведений: 

-1.407780

Соответствующий ему собственный вектор: 

DenseVector 3-Double
 0,696591
 0,121497
-0,707106


Фактическая погрешнсть: 

0.000000

index,AYminusLambdaY,AbsFactError,K,Lambda,PosteriorEstimation
0,1.1382245218511011,0.9639081870822439,1,-0.44387142759166404,0.7598378139876837
1,1.070811845196695,0.658877252708455,2,-0.7489023619654529,0.6992199829516441
2,0.6565653065148922,0.21467973296144516,3,-1.1930998817124627,0.32736770768428286
3,0.3885409372307627,0.04664082141588599,4,-1.361138793258022,0.13939030459827187
4,0.252519225724827,0.009715219621224458,5,-1.3980643950526834,0.06450081336878374
5,0.1693315760535562,0.002096543613842483,6,-1.4056830710600654,0.030737668359132952
6,0.11462602960328858,0.0004677220398423554,7,-1.4073118926340655,0.014781638870856434
7,0.0778855195239591,0.00010663854864745304,8,-1.4076729761252604,0.007134611962947052
8,0.05301616138745155,2.4627863571158315E-05,9,-1.4077549868103367,0.003449763040612405
9,0.036121241229673855,5.729388345310227E-06,10,-1.4077738852855626,0.0016695884910306422
